<a href="https://colab.research.google.com/github/kenken0830/Graphing/blob/main/articulated_animation_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# セットアップ

**githubからコードを取得**

In [2]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!sudo apt-get install git-lfs
!git lfs install
!git clone https://github.com/cedro3/articulated-animation.git
%cd articulated-animation

Detected operating system as Ubuntu/bionic.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... done.
Running apt-get update... done.

The repository is setup! You can now install packages.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (3.2.0).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.
Git LFS initialized.
fatal: destination path 'articulated-animation' already exists and is not an empty directory.
/content/articulated-animation


In [6]:
!pip install pyyaml==5.4.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 636 kB 8.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0


**学習済みパラメータのダウンロード**

In [3]:
! pip install --upgrade gdown
import gdown
gdown.download('https://drive.google.com/uc?id=1YHhLDf7QGhVUyAIMRsJBmCm0VDDcQdVV', './pt.zip', quiet=False)
! unzip pt.zip -d checkpoints

from demo import load_checkpoints
generator, region_predictor, avd_network = load_checkpoints(config_path='config/ted384.yaml',
                                                            checkpoint_path='checkpoints/ted384.pth')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Downloading...
From: https://drive.google.com/uc?id=1YHhLDf7QGhVUyAIMRsJBmCm0VDDcQdVV
To: /content/articulated-animation/pt.zip
100%|██████████| 254M/254M [00:00<00:00, 304MB/s]


Archive:  pt.zip
replace checkpoints/ted384.pth? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: checkpoints/ted384.pth  


/content/articulated-animation/demo.py:36: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(f)
/usr/local/lib/python3.7/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 静止画を動画に合わせてアニメーション

**静止画と動画の読み込み**

In [4]:
pwd

'/content/articulated-animation'

In [6]:
!pip install imageio-ffmpeg

A
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 26.9 MB 1.3 MB/s 


In [9]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

source_image = imageio.imread('sup-mat/02.jpg')
driving_video = imageio.mimread('sup-mat/movie.mp4')


source_image = resize(source_image, (384, 384))[..., :3]
driving_video = [resize(frame, (384, 384))[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
    

HTML(display(source_image, driving_video).to_html5_video())

**アニメーションの作成**

In [10]:
from demo import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, driving_video, generator, 
                             region_predictor, avd_network, animation_mode='avd')

#save resulting video
imageio.mimsave('generated.mp4', [img_as_ubyte(frame) for frame in predictions])
#video can be downloaded from /content folder

HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 429/429 [00:25<00:00, 16.80it/s]
